In [ ]:
!pip install kubernetes

In [ ]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print(i.status.conditions)
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

In [4]:
ret = v1.list_namespaced_pod("default")
pods = []
for i in ret.items:
  pods.append({
    'name': i.metadata.name,
    'namespace': i.metadata.namespace,
    'image':
    
  })
  print(i.status.conditions)


[{'last_probe_time': None,
 'last_transition_time': datetime.datetime(2022, 5, 6, 16, 50, 51, tzinfo=tzlocal()),
 'message': None,
 'reason': None,
 'status': 'True',
 'type': 'Initialized'}, {'last_probe_time': None,
 'last_transition_time': datetime.datetime(2022, 5, 6, 16, 51, 6, tzinfo=tzlocal()),
 'message': None,
 'reason': None,
 'status': 'True',
 'type': 'Ready'}, {'last_probe_time': None,
 'last_transition_time': datetime.datetime(2022, 5, 6, 16, 51, 6, tzinfo=tzlocal()),
 'message': None,
 'reason': None,
 'status': 'True',
 'type': 'ContainersReady'}, {'last_probe_time': None,
 'last_transition_time': datetime.datetime(2022, 5, 6, 16, 50, 50, tzinfo=tzlocal()),
 'message': None,
 'reason': None,
 'status': 'True',
 'type': 'PodScheduled'}]
[{'last_probe_time': None,
 'last_transition_time': datetime.datetime(2022, 5, 6, 16, 51, 31, tzinfo=tzlocal()),
 'message': None,
 'reason': None,
 'status': 'True',
 'type': 'Initialized'}, {'last_probe_time': None,
 'last_transition_ti

In [ ]:
ret = v1.list_pod_for_all_namespaces(watch=False)
ret.items[0]